In [2]:
import os 
import sys

In [3]:
os.getcwd()

'd:\\iNeuron\\Complete Project\\Boston-Regression\\Boston_Housing-Regression\\notebook'

In [4]:
os.chdir('..')

In [5]:
os.getcwd()

'd:\\iNeuron\\Complete Project\\Boston-Regression\\Boston_Housing-Regression'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    trained_model_path: Path

In [15]:
from src.utils import create_directories, read_yaml
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.logger import logging
from src.exception import CustomException

class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        try:
            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)
            create_directories([self.config.artifacts_root])
        except Exception as e:
            raise e
        
    def get_model_trainer_config(self):
        try:
            config=self.config.model_trainer

            trained_model_path = config.trained_model_path

            model_trainer_config=ModelTrainerConfig(
                trained_model_path=trained_model_path
            )

            return model_trainer_config
        except Exception as e:
            logging.info(e)
            raise CustomException(e,sys)

In [18]:
from src.logger import logging
from src.exception import CustomException
from src.utils import eval_metrics, save_obj

import pandas as pd
from pathlib import Path


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

class ModelTrainer:
    def __init__(self, config=ModelTrainerConfig):
        self.config=config

    def train_model(self):
        try:
            logging.info('Model Trainer Initiated')

            scaled_train_df = pd.read_csv(Path('artifact/data_transformation/scaled_train.csv'))
            scaled_test_df=pd.read_csv(Path('artifact/data_transformation/scaled_test.csv'))

            logging.info('Initiating Train test split')
            X_train = scaled_train_df.drop('MEDV', axis=1)
            y_train = scaled_train_df['MEDV']
            X_test = scaled_test_df.drop('MEDV', axis=1)
            y_test = scaled_test_df['MEDV']

            model = LinearRegression()
            model.fit(X_train, y_train) 
            logging.info('Model has been trained')

            trained_model_path = self.config.trained_model_path
            os.makedirs(os.path.dirname(trained_model_path), exist_ok=True)
            save_obj(trained_model_path,model)

            prediction = model.predict(X_test)

            mae, rmse, r2 = eval_metrics(y_test, prediction)

            logging.info(f"Linear Regression MAE: [{mae}], RMSE: [{rmse}], R2 Score: [{r2}]")
            return mae, rmse, r2
        except Exception as e:
            raise CustomException(e,sys)

In [19]:
if __name__=='__main__':
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer=ModelTrainer(config=model_trainer_config)
    model_trainer.train_model()